In [89]:
import pandas as pd
from joblib import dump
from nlpEngine import findKeywords 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [75]:
workbook = pd.read_excel('data.xlsx',sheet_name=None)
school_choice_responses = []
for name, sheet in workbook.items():
    for i, student in sheet.iterrows():
        response = None
        try:
            pathway = student['Based on your career aspirations, what pathway and course do you want to work towards?']
        except KeyError:
            pathway = student['Based on your career aspirations, what pathway do you want to work towards?']
        if pathway == 'Junior College':
            response = student['What is your 1st choice JC?']
        elif pathway == 'International Baccalaureate':
            response = student['What is your 1st choice IB course?']
        elif pathway == 'A Levels/International Baccalaureate':
            response = student['What is your 1st choice JC/IB?']
        elif pathway == 'NAFA/LaSalle':
            response = student['What is your 1st choice NAFA/LaSalle course?']
        elif pathway == 'Polytechnic':
            response =  student['What is your 1st choice polytechnic and course?']
        elif pathway == 'Polytechnic Foundation Programme (PFP)':
            response =  student['What is your 1st choice PFP course and polytechnic?']
        elif pathway == 'Direct to Polytechnic Programme (DPP)':
            response = student['What is your 1st choice Higher NITEC DPP course?']
        school_choice_responses.append(response)
school_choice_responses = pd.DataFrame(school_choice_responses,columns=['Response']).dropna()
school_choice_responses['Response'] = school_choice_responses['Response'].apply(findKeywords)
school_choice_responses['Response'] = school_choice_responses['Response'].apply(lambda x : ' '.join(x))
school_choice_responses


,Response
0,diploma engineering business
1,raffle institution jc
2,np accountancy
3,singapore poly experience communication n design
4,ngee ann polytechnic biomedical research
...,...
1118,design merchandise singapore poly
1119,business course
1120,apparel design merchandising
1121,nursing


In [76]:
matrix = CountVectorizer(max_features=1000)
X = matrix.fit_transform(school_choice_responses['Response']).toarray()

In [77]:
y = []
workbook = pd.read_excel('processedData.xlsx',sheet_name=None)
for name, sheet in workbook.items():
    for i, student in sheet.iterrows():
        if student['Pathway'] != 'Progress to Sec 5':
            school = student['School']
            y.append(school)


In [78]:
schools = ['Anglo-Chinese JC','Anderson Serangonn JC','Catholic JC','Dunman High School','Eunoia JC','Hwa Chong Institution','Jurong Pioneer JC','Millenia Institude','Nanyang JC','National JC','Raffles Institution','River Valley High School','St Andrews JC','St Josephs Insitution','Tampines Meridian JC','Temasek JC','Victoria JC','Yishun Innova JC','Ngee Ann Polytechnic','Singapore Polytechnic','Temasek Polytechnic','Nanyang Polytechnic','Republic Polytechnic']
new_X = []
new_y = []
for i in range(len(y)):
    if y[i] in schools:
        new_X.append(X[i])
        new_y.append(y[i])
X = new_X
y = new_y

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [79]:
model = GaussianNB()
model.fit(X_train,y_train)

prediction = model.predict(X_test)
print('Accuracy: ',accuracy_score(y_test,prediction))

Accuracy:  0.9615384615384616


In [80]:
model = RandomForestClassifier()
model.fit(X_train,y_train)

prediction = model.predict(X_test)
print('Accuracy: ',accuracy_score(y_test,prediction))

Accuracy:  0.9711538461538461


In [88]:
user_response = input()
user_response = matrix.transform([user_response]).toarray()
user_prediction = model.predict(user_response)
print(user_prediction)

['River Valley High School']


In [91]:
dump(matrix,'school_vectorizer.sav')
dump(model,'school_classifier.sav')

['school_classifier.sav']